### Import Libraries

In [1]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras import layers

## Create ML Models, Train Them, and Make Predictions

First, we prepare our data for training

In [2]:
train_data = pd.read_csv('prepared_data/train_data.csv')
test_data = pd.read_csv('prepared_data/test_data.csv')

In [3]:
target = 'charges'

train_labels = train_data[target].copy()
test_labels = test_data[target].copy()

# Drop 'charges' column
train_set = train_data.drop(target, axis=1)
test_set = test_data.drop(target, axis=1)

Here are the classification models we are using to make predictions:
* Linear Regression
* Random Forest Regressor
* XGBoost Regressor
* Neural Network

Now, we set them up with default parameters.

In [5]:
lr = LinearRegression()
rf = RandomForestRegressor(n_estimators=100, random_state=42)
xgb = XGBRegressor(n_estimators=200, learning_rate=0.1, random_state=42)
nn = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(train_set.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])
nn.compile(optimizer='adam', loss='mae')

c:\Users\nguye\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Then, we fit the models with the data we prepared.

In [6]:
lr.fit(train_set, train_labels)
rf.fit(train_set, train_labels)
xgb.fit(train_set, train_labels)
nn.fit(train_set, train_labels, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 13214.7051 - val_loss: 11957.2969
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 12405.9844 - val_loss: 11214.9307
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 11727.6172 - val_loss: 10215.6904
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 10865.2695 - val_loss: 9361.9004
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 9704.4375 - val_loss: 8855.2236
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 9526.1504 - val_loss: 8863.5537
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 9478.0205 - val_loss: 8885.8555
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 9218.8838 - val_loss: 8763.7041
Epoch 9/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 9711.0068 - val_loss: 8657.1250
Epoch 10/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 8842.8730 - val_loss: 8531.1406
Epoch 11/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 8380.4785 - val_loss: 8407.2